In this code, we are building a deep learning model using the Xception architecture as a feature extractor, followed by an SVM (Support Vector Machine) for classification. Initially, we load the pre-trained Xception model (without the top classification layer) and apply global average pooling on the output to reduce its dimensionality. Then, we create a new output layer with 2048 units and a ReLU activation function, which serves as an intermediate feature representation.

We then train the model to extract these features from images, specifically categorizing images as either 'Necrosis' or 'Non_Necrosis.' The features from the Xception model are saved as CSV files, along with the image labels and filenames. These features are then used to train an SVM classifier, which is evaluated based on performance metrics such as precision, recall, F1 score, accuracy, sensitivity, specificity, and AUC-ROC.

For multiple batches, the code processes training and testing image data, extracts features using Xception, trains the SVM model, and stores the evaluation results in CSV files. The overall goal is to combine deep learning feature extraction with a traditional machine learning classifier (SVM) to detect and classify images based on the presence of necrosis.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import zipfile
import os

# Get the current notebook's directory and navigate to the correct location
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Go up to first pathologyStudentsAug25
parent_dir = os.path.dirname(current_dir)
# Go up to Downloads
root_dir = os.path.dirname(parent_dir)
# Construct path to zip file
path_to_zip_file = os.path.join(root_dir, 'pathologyStudentsAug25', 'necrosisData', 'WR_Necrosis 3.0.zip')

# Create extraction directory
extract_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'WR_Necrosis 3.0')
os.makedirs(extract_dir, exist_ok=True)

print(f"Looking for zip file at: {path_to_zip_file}")
print(f"Will extract to: {extract_dir}")

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

Looking for zip file at: /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/necrosisData/WR_Necrosis 3.0.zip
Will extract to: /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/WR_Necrosis 3.0


In [3]:
import zipfile
import os

# Get the current notebook's directory and navigate to the correct location
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Go up to first pathologyStudentsAug25
parent_dir = os.path.dirname(current_dir)
# Go up to Downloads
root_dir = os.path.dirname(parent_dir)
# Construct path to zip file
path_to_zip_file = os.path.join(root_dir, 'pathologyStudentsAug25', 'necrosisData', 'WR_Non Necrosis 3.0.zip')

# Create extraction directory
extract_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'WR_Non_Necrosis 3.0')
os.makedirs(extract_dir, exist_ok=True)

print(f"Looking for zip file at: {path_to_zip_file}")
print(f"Will extract to: {extract_dir}")

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

Looking for zip file at: /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/necrosisData/WR_Non Necrosis 3.0.zip
Will extract to: /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/WR_Non_Necrosis 3.0


In [5]:
import glob
import shutil
import os

In [2]:
def batch_preparation(batch_number,N_train,NN_train,N_test,NN_test,N_train_per,NN_train_per,N_test_per,NN_test_per):
  # Get the current notebook's directory and navigate to the correct location
  current_dir = os.path.dirname(os.path.abspath('__file__'))
  parent_dir = os.path.dirname(current_dir)
  root_dir = os.path.dirname(parent_dir)
  
  # Create batch directory structure
  batch_dir = os.path.join(root_dir, 'pathologyStudentsAug25','original', f'Batch_{batch_number}')
  os.makedirs(os.path.join(batch_dir, 'Training', 'Necrosis'), exist_ok=True)
  os.makedirs(os.path.join(batch_dir, 'Training', 'Non_Necrosis'), exist_ok=True)
  os.makedirs(os.path.join(batch_dir, 'Testing', 'Necrosis'), exist_ok=True)
  os.makedirs(os.path.join(batch_dir, 'Testing', 'Non_Necrosis'), exist_ok=True)

  # Define source directories
  necrosis_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'WR_Necrosis 3.0')
  non_necrosis_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'WR_Non_Necrosis 3.0')
  runtime_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Run_time_Folder')

  for i in range(len(N_train)):
    N_train[i]=str(N_train[i])
    if N_train_per[i]==1:
      print('Processing Necrosis in Training for Sample Number {} '.format(N_train[i]))
      print('\n')
      src_dir = os.path.join(necrosis_dir, 'Sample ' + N_train[i])
      dst_dir = os.path.join(batch_dir, 'Training', 'Necrosis')
      for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
    else:
      print('Processing Necrosis in Testing for Sample Number {}'.format(N_train[i]))
      print('\n')
      src_dir = os.path.join(necrosis_dir, 'Sample ' + N_train[i])
      files_names = os.listdir(src_dir)
      files_count = len(files_names)
      selected_number_of_files = int(N_train_per[i]*files_count)
      selected_files = files_names[0:selected_number_of_files]
      os.makedirs(runtime_dir, exist_ok=True)
      for j in range(selected_number_of_files):
        shutil.copy(os.path.join(src_dir, selected_files[j]), runtime_dir)
      dst_dir = os.path.join(batch_dir, 'Training', 'Necrosis')
      for jpgfile in glob.iglob(os.path.join(runtime_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
      shutil.rmtree(runtime_dir, ignore_errors=True)

  for i in range(len(NN_train)):
    NN_train[i]=str(NN_train[i])
    if NN_train_per[i]==1:
      print('Processing Non_Necrosis in Training for Sample Number {}'.format(NN_train[i]))
      print('\n')
      src_dir = os.path.join(non_necrosis_dir, 'Sample ' + NN_train[i])
      dst_dir = os.path.join(batch_dir, 'Training', 'Non_Necrosis')
      for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
    else:
      print('Processing Non_Necrosis in Testing for Sample Number {}'.format(NN_train[i]))
      print('\n')
      src_dir = os.path.join(non_necrosis_dir, 'Sample ' + NN_train[i])
      files_names = os.listdir(src_dir)
      files_count = len(files_names)
      selected_number_of_files = int(NN_train_per[i]*files_count)
      selected_files = files_names[0:selected_number_of_files]
      os.makedirs(runtime_dir, exist_ok=True)
      for j in range(selected_number_of_files):
        shutil.copy(os.path.join(src_dir, selected_files[j]), runtime_dir)
      dst_dir = os.path.join(batch_dir, 'Training', 'Non_Necrosis')
      for jpgfile in glob.iglob(os.path.join(runtime_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
      shutil.rmtree(runtime_dir, ignore_errors=True)

  for i in range(len(N_test)):
    N_test[i]=str(N_test[i])
    if N_test_per[i]==1:
      print('Processing Necrosis in Testing for Sample Number {}'.format(N_test[i]))
      print('\n')
      src_dir = os.path.join(necrosis_dir, 'Sample ' + N_test[i])
      dst_dir = os.path.join(batch_dir, 'Testing', 'Necrosis')
      for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
    else:
      print('Processing Necrosis in Testing for Sample Number {}'.format(N_test[i]))
      print('\n')
      src_dir = os.path.join(necrosis_dir, 'Sample ' + N_test[i])
      files_names = os.listdir(src_dir)
      files_count = len(files_names)
      selected_number_of_files = int(N_test_per[i]*files_count)
      selected_files = files_names[selected_number_of_files:files_count]
      os.makedirs(runtime_dir, exist_ok=True)
      for j in range(selected_number_of_files):
        shutil.copy(os.path.join(src_dir, selected_files[j]), runtime_dir)
      dst_dir = os.path.join(batch_dir, 'Testing', 'Necrosis')
      for jpgfile in glob.iglob(os.path.join(runtime_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
      shutil.rmtree(runtime_dir, ignore_errors=True)

  for i in range(len(NN_test)):
    NN_test[i]=str(NN_test[i])
    if NN_test_per[i]==1:
      print('Processing Non Necrosis in Testing for Sample Number {}'.format(NN_test[i]))
      print('\n')
      src_dir = os.path.join(non_necrosis_dir, 'Sample ' + NN_test[i])
      dst_dir = os.path.join(batch_dir, 'Testing', 'Non_Necrosis')
      for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
    else:
      print('Processing Non Necrosis in Testing for Sample Number {}'.format(NN_test[i]))
      src_dir = os.path.join(non_necrosis_dir, 'Sample ' + NN_test[i])
      files_names = os.listdir(src_dir)
      files_count = len(files_names)
      selected_number_of_files = int(NN_test_per[i]*files_count)
      selected_files = files_names[selected_number_of_files:files_count]
      os.makedirs(runtime_dir, exist_ok=True)
      for j in range(selected_number_of_files):
        shutil.copy(os.path.join(src_dir, selected_files[j]), runtime_dir)
      dst_dir = os.path.join(batch_dir, 'Testing', 'Non_Necrosis')
      for jpgfile in glob.iglob(os.path.join(runtime_dir, "*.png")):
        shutil.copy(jpgfile, dst_dir)
      shutil.rmtree(runtime_dir, ignore_errors=True)

In [3]:
Batches=[1,2,3,4,5,6,7]


B1_Train_N=[7,6,4]
B1_Train_Per=[0.8,0.8,0.8]
B1_Train_NN=[2,4,5,6,8,9]
B1_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B1_Test_N=[2]
B1_Test_Per=[0.2]
B1_Test_NN=[1]
B1_Test_NN_Per=[0.2]

B2_Train_N=[7,6,2]
B2_Train_Per=[0.8,0.8,0.8]
B2_Train_NN=[2,1,4,6,8,9]
B2_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B2_Test_N=[4]
B2_Test_Per=[0.2]
B2_Test_NN=[5]
B2_Test_NN_Per=[0.2]

B3_Train_N=[7,4,2]
B3_Train_Per=[0.8,0.8,0.8]
B3_Train_NN=[2,4,5,9,8,1]
B3_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B3_Test_N=[6]
B3_Test_Per=[0.2]
B3_Test_NN=[6]
B3_Test_NN_Per=[0.2]

B4_Train_N=[7,6,4,2]
B4_Train_Per=[0.8,0.8,0.8,0.8]
B4_Train_NN=[2,4,5,6,1,9]
B4_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B4_Test_N=[7]
B4_Test_Per=[0.2]
B4_Test_NN=[8]
B4_Test_NN_Per=[0.2]

B5_Train_N=[7,6,4,2]
B5_Train_Per=[0.8,0.8,0.8,0.8]
B5_Train_NN=[2,4,5,6,8,9,1]
B5_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8,0.8]
B5_Test_N=[7]
B5_Test_Per=[0.2]
B5_Test_NN=[2]
B5_Test_NN_Per=[0.2]


B6_Train_N=[7,6,4]
B6_Train_Per=[0.8,0.8,0.8]
B6_Train_NN=[2,4,5,6,9,1,8]
B6_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8,0.8]
B6_Test_N=[2]
B6_Test_Per=[0.2]
B6_Test_NN=[4]
B6_Test_NN_Per=[0.2]

B7_Train_N=[7,6,4]
B7_Train_Per=[0.8,0.8,0.8]
B7_Train_NN=[2,4,5,9,1,8]
B7_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B7_Test_N=[2]
B7_Test_Per=[0.2]
B7_Test_NN=[6]
B7_Test_NN_Per=[0.2]

Batches=[8,9,10,11,12,13,14,15]

B8_Train_N=[2,4,7]
B8_Train_Per=[0.8,0.8,0.8]
B8_Train_NN=[1,2,4,5,8,9]
B8_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B8_Test_N=[6]
B8_Test_Per=[0.2]
B8_Test_NN=[6]
B8_Test_NN_Per=[0.2]



B9_Train_N=[6,4,7]
B9_Train_Per=[0.8,0.8,0.8]
B9_Train_NN=[1,4,5,8,9,6]
B9_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B9_Test_N=[2]
B9_Test_Per=[0.2]
B9_Test_NN=[2]
B9_Test_NN_Per=[0.2]


B10_Train_N=[6,2,7]
B10_Train_Per=[0.8,0.8,0.8]
B10_Train_NN=[1,2,5,8,9,6]
B10_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B10_Test_N=[4]
B10_Test_Per=[0.2]
B10_Test_NN=[4]
B10_Test_NN_Per=[0.2]

B11_Train_N=[6,2,4]
B11_Train_Per=[0.8,0.8,0.8]
B11_Train_NN=[4,2,5,8,9,6]
B11_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B11_Test_N=[7]
B11_Test_Per=[0.2]
B11_Test_NN=[1]
B11_Test_NN_Per=[0.2]


B12_Train_N=[6,2,7]
B12_Train_Per=[0.8,0.8,0.8]
B12_Train_NN=[1,2,5,9,6]
B12_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8]
B12_Test_N=[4]
B12_Test_Per=[0.2]
B12_Test_NN=[8]
B12_Test_NN_Per=[0.2]


B13_Train_N=[6,2,7]
B13_Train_Per=[0.8,0.8,0.8]
B13_Train_NN=[1,2,5,8,6]
B13_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8]
B13_Test_N=[4]
B13_Test_Per=[0.2]
B13_Test_NN=[9]
B13_Test_NN_Per=[0.2]


B14_Train_N=[6,2,7]
B14_Train_Per=[0.8,0.8,0.8]
B14_Train_NN=[9,2,5,8,6]
B14_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8,0.8]
B14_Test_N=[4]
B14_Test_Per=[0.2]
B14_Test_NN=[1]
B14_Test_NN_Per=[0.2]


B15_Train_N=[6,2,7]
B15_Train_Per=[0.8,0.8,0.8]
B15_Train_NN=[1,9,2,8,6]
B15_Train_NN_Per=[0.8,0.8,0.8,0.8,0.8]
B15_Test_N=[4]
B15_Test_Per=[0.2]
B15_Test_NN=[5]
B15_Test_NN_Per=[0.2]

B16_Train_N=[7,6,4]
B16_Train_Per=[1,1,1]
B16_Train_NN=[1,4,5,6,8,9]
B16_Train_NN_Per=[1,1,1,1,1,1]
B16_Test_N=[2]
B16_Test_Per=[1]
B16_Test_NN=[2]
B16_Test_NN_Per=[1]

B17_Train_N=[7,6,2]
B17_Train_Per=[1,1,1]
B17_Train_NN=[1,2,5,6,8,9]
B17_Train_NN_Per=[1,1,1,1,1,1]
B17_Test_N=[4]
B17_Test_Per=[1]
B17_Test_NN=[4]
B17_Test_NN_Per=[1]

B18_Train_N=[2,4,7]
B18_Train_Per=[1,1,1]
B18_Train_NN=[1,2,4,5,8,9]
B18_Train_NN_Per=[1,1,1,1,1,1]
B18_Test_N=[6]
B18_Test_Per=[1]
B18_Test_NN=[6]
B18_Test_NN_Per=[1]

B19_Train_N=[2,4,6]
B19_Train_Per=[1,1,1]
B19_Train_NN=[1,2,4,5,8,9]
B19_Train_NN_Per=[1,1,1,1,1,1]
B19_Test_N=[7]
B19_Test_Per=[0.2]
B19_Test_NN=[7]
B19_Test_NN_Per=[1]

In [6]:
batch_preparation(1,B1_Train_N,B1_Train_NN,B1_Test_N,B1_Test_NN,B1_Train_Per,B1_Train_NN_Per,B1_Test_Per,B1_Test_NN_Per)
batch_preparation(2,B2_Train_N,B2_Train_NN,B2_Test_N,B2_Test_NN,B2_Train_Per,B2_Train_NN_Per,B2_Test_Per,B2_Test_NN_Per)
batch_preparation(3,B3_Train_N,B3_Train_NN,B3_Test_N,B3_Test_NN,B3_Train_Per,B3_Train_NN_Per,B3_Test_Per,B3_Test_NN_Per)
batch_preparation(4,B4_Train_N,B4_Train_NN,B4_Test_N,B4_Test_NN,B4_Train_Per,B4_Train_NN_Per,B4_Test_Per,B4_Test_NN_Per)
batch_preparation(5,B5_Train_N,B5_Train_NN,B5_Test_N,B5_Test_NN,B5_Train_Per,B5_Train_NN_Per,B5_Test_Per,B5_Test_NN_Per)
batch_preparation(6,B6_Train_N,B6_Train_NN,B6_Test_N,B6_Test_NN,B6_Train_Per,B6_Train_NN_Per,B6_Test_Per,B6_Test_NN_Per)
batch_preparation(7,B7_Train_N,B7_Train_NN,B7_Test_N,B7_Test_NN,B7_Train_Per,B7_Train_NN_Per,B7_Test_Per,B7_Test_NN_Per)
batch_preparation(8,B8_Train_N,B8_Train_NN,B8_Test_N,B8_Test_NN,B8_Train_Per,B8_Train_NN_Per,B8_Test_Per,B8_Test_NN_Per)
batch_preparation(9,B9_Train_N,B9_Train_NN,B9_Test_N,B9_Test_NN,B9_Train_Per,B9_Train_NN_Per,B9_Test_Per,B9_Test_NN_Per)
batch_preparation(10,B10_Train_N,B10_Train_NN,B10_Test_N,B10_Test_NN,B10_Train_Per,B10_Train_NN_Per,B10_Test_Per,B10_Test_NN_Per)
batch_preparation(11,B11_Train_N,B11_Train_NN,B11_Test_N,B11_Test_NN,B11_Train_Per,B11_Train_NN_Per,B11_Test_Per,B11_Test_NN_Per)
batch_preparation(12,B12_Train_N,B12_Train_NN,B12_Test_N,B12_Test_NN,B12_Train_Per,B12_Train_NN_Per,B12_Test_Per,B12_Test_NN_Per)
batch_preparation(13,B13_Train_N,B13_Train_NN,B13_Test_N,B13_Test_NN,B13_Train_Per,B13_Train_NN_Per,B13_Test_Per,B13_Test_NN_Per)
batch_preparation(14,B14_Train_N,B14_Train_NN,B14_Test_N,B14_Test_NN,B14_Train_Per,B14_Train_NN_Per,B14_Test_Per,B14_Test_NN_Per)
batch_preparation(15,B15_Train_N,B15_Train_NN,B15_Test_N,B15_Test_NN,B15_Train_Per,B15_Train_NN_Per,B15_Test_Per,B15_Test_NN_Per)
batch_preparation(16,B16_Train_N,B16_Train_NN,B16_Test_N,B16_Test_NN,B16_Train_Per,B16_Train_NN_Per,B16_Test_Per,B16_Test_NN_Per)
batch_preparation(17,B17_Train_N,B17_Train_NN,B17_Test_N,B17_Test_NN,B17_Train_Per,B17_Train_NN_Per,B17_Test_Per,B17_Test_NN_Per)
batch_preparation(18,B18_Train_N,B18_Train_NN,B18_Test_N,B18_Test_NN,B18_Train_Per,B18_Train_NN_Per,B18_Test_Per,B18_Test_NN_Per)
batch_preparation(19,B19_Train_N,B19_Train_NN,B19_Test_N,B19_Test_NN,B19_Train_Per,B19_Train_NN_Per,B19_Test_Per,B19_Test_NN_Per)

Processing Necrosis in Testing for Sample Number 7


Processing Necrosis in Testing for Sample Number 6


Processing Necrosis in Testing for Sample Number 4


Processing Non_Necrosis in Testing for Sample Number 2


Processing Non_Necrosis in Testing for Sample Number 4


Processing Non_Necrosis in Testing for Sample Number 5


Processing Non_Necrosis in Testing for Sample Number 6


Processing Non_Necrosis in Testing for Sample Number 8


Processing Non_Necrosis in Testing for Sample Number 9


Processing Necrosis in Testing for Sample Number 2


Processing Non Necrosis in Testing for Sample Number 1
Processing Necrosis in Testing for Sample Number 7


Processing Necrosis in Testing for Sample Number 6


Processing Necrosis in Testing for Sample Number 2


Processing Non_Necrosis in Testing for Sample Number 2


Processing Non_Necrosis in Testing for Sample Number 1


Processing Non_Necrosis in Testing for Sample Number 4


Processing Non_Necrosis in Testing for Sample Number 6


Proc

In [1]:
# Import basic libraries
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
import cv2
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage.feature import hog
import skimage.feature as feature

# Machine learning
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, f1_score, precision_score,
    cohen_kappa_score, matthews_corrcoef, roc_auc_score,
    confusion_matrix
)

# Deep learning
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import (
    DenseNet169, Xception, MobileNet, ResNet50, DenseNet121,
    EfficientNetB0, VGG16, MobileNetV2, ResNet101,
    InceptionResNetV2, InceptionV3, NASNetMobile
)
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten,
    Dropout, BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)

# Enable eager execution for TensorFlow
tf.config.run_functions_eagerly(True)
# =========================================
# FIX 2: Enable dynamic GPU memory allocation
# =========================================
tf.keras.backend.clear_session()

# Enable dynamic memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:        # <-- FIXED HERE
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except Exception as e:
        print("Memory growth error:", e)
else:
    print("No GPU detected.")

# Create the Xception model with feature extraction
base_model_Xception = Xception(
    input_shape=(256, 256, 3),
    weights='imagenet',
    include_top=False
)

# Add global average pooling
output_1 = GlobalAveragePooling2D()(base_model_Xception.output)

# Add dense layer for feature extraction
output_4 = Dense(2048, activation='relu')(output_1)
model_Xception_features_2048 = Model(base_model_Xception.input, output_4)

# Add final classification layer
output = Dense(2, activation='softmax')(output_1)

2025-12-02 22:40:51.510129: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 22:40:51.560411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 22:40:53.214147: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


GPU memory growth enabled.


I0000 00:00:1764695454.077116 4187467 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1013 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:04:00.0, compute capability: 8.6


In [2]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import gc

# =========================================
# 🧠 GPU MEMORY MANAGEMENT
# =========================================
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')   # reduce memory by ~40%

# =========================================
# 🧩 GLOBAL MODEL CACHE
# =========================================
_cached_model = None


# =============================================================
# 1️⃣ BUILD FEATURE EXTRACTOR (WITH TRAINABLE RELU LAYER)
# =============================================================
def build_feature_extractor_with_relu():
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(256, 256, 3)
    )

    # Freeze the Xception base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add trainable Dense(2048, relu)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(2048, activation='relu', dtype='float32', name="feature_layer",trainable=True)(x)

    output = Dense(2, activation='softmax', dtype='float32')(x)

    model = Model(inputs=base_model.input, outputs=output)
    print("[INFO] Model created → Base frozen + Trainable Dense(2048, ReLU)")
    return model


# =============================================================
# 2️⃣ TRAIN ONLY THE DENSE(2048, RELU) LAYER
# =============================================================
def train_dense_layer(model, train_generator, val_generator, epochs=3):
    """Train the trainable Dense layer to learn better features."""

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\n[TRAINING] 🔥 Training Dense(2048, ReLU) layer …")

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs
    )

    print("[TRAINING] ✅ Dense layer training finished!\n")
    return model


# =============================================================
# 3️⃣ GET CACHED MODEL OR TRAIN NEW ONE
# =============================================================
def get_feature_extraction_model(train_gen=None, val_gen=None):
    global _cached_model

    if _cached_model is None:
        print("[INFO] Building model for first time…")

        # Build model
        model = build_feature_extractor_with_relu()

        # Train Dense layer (only once)
        if train_gen is not None and val_gen is not None:
            model = train_dense_layer(model, train_gen, val_gen, epochs=3)

        trained_feature_model = Model(
            inputs=model.input,
            outputs=model.get_layer("feature_layer").output
        )

        feature_save_path = os.path.join("saved_models", "Xception_feature_extractor_original.h5")
        trained_feature_model.save(feature_save_path)

        _cached_model = trained_feature_model
        print("[INFO] ✅ Feature extraction model cached")

    return _cached_model


# =============================================================
# 4️⃣ CHUNK-BASED FEATURE EXTRACTION
# =============================================================
def extract_features_in_chunks(generator, model, chunk_size=32):
    all_features = []
    all_labels = []
    all_filenames = []

    total_samples = len(generator.filenames)
    steps = int(np.ceil(total_samples / generator.batch_size))

    print(f"[INFO] Extracting {total_samples} images in {steps} steps…")

    for step in range(steps):
        try:
            batch_x, batch_y = next(generator)
            batch_features = model.predict(batch_x, verbose=0)

            all_features.append(batch_features)
            all_labels.extend(np.argmax(batch_y, axis=1))
            all_filenames.extend(generator.filenames[step*generator.batch_size:(step+1)*generator.batch_size])

            if (step + 1) % 10 == 0:
                print(f"  [Progress] {step + 1}/{steps}")
                gc.collect()

        except StopIteration:
            break

    features = np.vstack(all_features)
    print(f"[INFO] ✅ Final features shape: {features.shape}")
    return features, np.array(all_labels), all_filenames[:len(all_labels)]


# =============================================================
# 5️⃣ MAIN EXTRACTION FUNCTION
# =============================================================
def XceptionNet2048_original_Features_Optimized(train_generator, test_generator, Batch):

    # TRAINING DIRECTORIES FOR DENSE LAYER
    # (You MUST create separate augmentation generators)
    train_aug = ImageDataGenerator(rescale=1/255, rotation_range=20, horizontal_flip=True)
    val_aug = ImageDataGenerator(rescale=1/255)

    train_gen_dense = train_aug.flow_from_directory(
        train_generator.directory,
        target_size=(256, 256),
        batch_size=16,
        class_mode='categorical',
        shuffle=True
    )

    val_gen_dense = val_aug.flow_from_directory(
        test_generator.directory,
        target_size=(256, 256),
        batch_size=16,
        class_mode='categorical',
        shuffle=True
    )

    # Load or build model
    model = get_feature_extraction_model(train_gen_dense, val_gen_dense)

    # Create output directory
    current_dir = os.path.dirname(os.path.abspath('__file__'))
    parent_dir = os.path.dirname(current_dir)
    root_dir = os.path.dirname(parent_dir)
    save_drive_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Results', 'XceptionNet2048+SVM')
    batch_output_dir = os.path.join(save_drive_dir, Batch)
    os.makedirs(batch_output_dir, exist_ok=True)

    # EXTRACT FEATURES
    print(f"\n[STEP 1] Extracting training features for {Batch}")
    f_train, y_train, n_train = extract_features_in_chunks(train_generator, model)

    print(f"\n[STEP 2] Extracting test features for {Batch}")
    f_test, y_test, n_test = extract_features_in_chunks(test_generator, model)

    # SAVE FEATURES
    cols = [f"feature_{i}" for i in range(f_train.shape[1])]

    df_train = pd.DataFrame(f_train, columns=cols)
    df_train["label"] = y_train
    df_train["Image_Name"] = n_train
    df_train.to_csv(os.path.join(batch_output_dir, "XceptionNet2048_Training.csv"), index=False)

    df_test = pd.DataFrame(f_test, columns=cols)
    df_test["label"] = y_test
    df_test["Image_Name"] = n_test
    df_test.to_csv(os.path.join(batch_output_dir, "XceptionNet2048_Testing.csv"), index=False)

    print(f"[DONE] Features saved for {Batch}\n")


# =============================================================
# 6️⃣ PROCESS ALL BATCHES
# =============================================================
print("\n" + "="*60)
print("🚀 STARTING OPTIMIZED FEATURE EXTRACTION (ReLU + TRAINED)")
print("="*60 + "\n")

for x in range(16, 20):

    batch_size = 32
    Batch_Name = f"Batch_{x}"

    current_dir = os.path.dirname(os.path.abspath('__file__'))
    parent_dir = os.path.dirname(current_dir)
    root_dir = os.path.dirname(parent_dir)

    Training_Path = os.path.join(root_dir, "pathologyStudentsAug25", "original", Batch_Name, "Training")
    Testing_Path = os.path.join(root_dir, "pathologyStudentsAug25", "original", Batch_Name, "Testing")

    if not os.path.exists(Training_Path):
        print(f"[SKIP] Missing path: {Training_Path}")
        continue

    train_datagen = ImageDataGenerator(rescale=1/255)
    test_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(
        Training_Path,
        target_size=(256, 256),
        batch_size=batch_size,
        shuffle=False,
        classes=["Non_Necrosis", "Necrosis"]
    )

    test_generator = test_datagen.flow_from_directory(
        Testing_Path,
        target_size=(256, 256),
        batch_size=batch_size,
        shuffle=False,
        classes=["Non_Necrosis", "Necrosis"]
    )

    XceptionNet2048_original_Features_Optimized(train_generator, test_generator, Batch_Name)


print("\n" + "="*60)
print("🎉 ALL BATCHES PROCESSED WITH TRAINED RELU FEATURES!")
print("="*60)



🚀 STARTING OPTIMIZED FEATURE EXTRACTION (ReLU + TRAINED)

Found 4586 images belonging to 2 classes.


Found 3514 images belonging to 2 classes.
Found 4586 images belonging to 2 classes.
Found 3514 images belonging to 2 classes.
[INFO] Building model for first time…
[INFO] Model created → Base frozen + Trainable Dense(2048, ReLU)

[TRAINING] 🔥 Training Dense(2048, ReLU) layer …
Epoch 1/3


2025-12-02 22:53:32.272143: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2025-12-02 22:53:32.312993: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-12-02 22:53:34.286285: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 260.16MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-02 22:53:44.503283: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 122.07MiB (rounded to 128000000)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current 

ResourceExhaustedError: Exception encountered when calling BatchNormalization.call().

[1m{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: [0m

Arguments received by BatchNormalization.call():
  • inputs=tf.Tensor(shape=(16, 125, 125, 128), dtype=float16)
  • training=True
  • mask=None

In [7]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D
import gc

# =========================================
# 🧠 GPU MEMORY MANAGEMENT
# =========================================
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# Use mixed precision (reduces memory by ~40%)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# =========================================
# 🧩 GLOBAL MODEL CACHE - REUSE ACROSS BATCHES
# =========================================
_cached_model = None

def get_feature_extraction_model():
    """Get cached or create new model for feature extraction"""
    global _cached_model
    if _cached_model is None:
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
        output_pooled = GlobalAveragePooling2D()(base_model.output)
        output_4 = Dense(2048, activation='relu')(output_pooled)
        _cached_model = Model(inputs=base_model.input, outputs=output_4)
        
        # Freeze all layers
        for layer in base_model.layers:
            layer.trainable = False
        print("[INFO] ✅ Feature extraction model loaded (cached)")
    return _cached_model

# =========================================
# 🧮 OPTIMIZED FEATURE EXTRACTION - FULL DATA SUPPORT
# =========================================
def extract_features_in_chunks(generator, model, chunk_size=32):
    """
    Extract features in chunks to manage memory efficiently.
    Handles full dataset regardless of size.
    """
    all_features = []
    all_labels = []
    all_filenames = []
    
    # Calculate total steps needed
    total_samples = len(generator.filenames)
    steps = int(np.ceil(total_samples / generator.batch_size))
    
    print(f"[INFO] Extracting features from {total_samples} images in {steps} steps...")
    
    for step in range(steps):
        try:
            batch_x, batch_y = next(generator)
            chunk_features = model.predict(batch_x, verbose=0)
            all_features.append(chunk_features)
            all_labels.extend(batch_y if isinstance(batch_y, (list, np.ndarray)) and len(batch_y.shape) == 1 
                            else np.argmax(batch_y, axis=1))
            all_filenames.extend(generator.filenames[step*generator.batch_size:(step+1)*generator.batch_size])
            
            if (step + 1) % 10 == 0:
                print(f"  [Progress] {step + 1}/{steps} batches processed...")
                gc.collect()
        except StopIteration:
            break
    
    # Concatenate all features
    features_array = np.vstack(all_features)
    print(f"[INFO] ✅ Total features extracted: {features_array.shape}")
    
    return features_array, np.array(all_labels), all_filenames[:len(all_labels)]

# =========================================
# 🚀 MAIN OPTIMIZED FUNCTION - FEATURE EXTRACTION ONLY
# =========================================
def XceptionNet2048_Features_Optimized(train_generator, test_generator, Batch):
    """
    Optimized feature extraction (100% data support).
    - Supports 100% data processing (not just 40%)
    - Memory-efficient chunked processing
    - Cached model across batches
    - Saves features to CSV
    """
    
    # Setup paths
    current_dir = os.path.dirname(os.path.abspath('__file__'))
    parent_dir = os.path.dirname(current_dir)
    root_dir = os.path.dirname(parent_dir)
    save_drive_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Results', 'XceptionNet2048+SVM')
    batch_output_dir = os.path.join(save_drive_dir, Batch)
    os.makedirs(batch_output_dir, exist_ok=True)
    
    print(f"\n{'='*60}")
    print(f"[BATCH] Processing {Batch}")
    print(f"{'='*60}")
    
    # Get cached model
    model = get_feature_extraction_model()
    
    # ✅ FEATURE EXTRACTION - FULL DATA (100%)
    print(f"\n[STEP 1/2] Extracting training features...")
    features_train, labels_train, filenames_train = extract_features_in_chunks(
        train_generator, model, chunk_size=32
    )
    
    print(f"\n[STEP 1/2] Extracting testing features...")
    features_test, labels_test, filenames_test = extract_features_in_chunks(
        test_generator, model, chunk_size=32
    )
    
    # ✅ SAVE RAW FEATURES
    columns_2048 = [f'feature_{c}' for c in range(features_train.shape[1])]
    
    df_train = pd.DataFrame(features_train, columns=columns_2048)
    df_train['label'] = labels_train
    df_train['Image_Name'] = filenames_train
    df_train.to_csv(os.path.join(batch_output_dir, 'XceptionNet2048_Training.csv'), index=False)
    print(f"[STEP 2/2] ✅ Training features saved: {df_train.shape[0]} samples")
    
    df_test = pd.DataFrame(features_test, columns=columns_2048)
    df_test['label'] = labels_test
    df_test['Image_Name'] = filenames_test
    df_test.to_csv(os.path.join(batch_output_dir, 'XceptionNet2048_Testing.csv'), index=False)
    print(f"[STEP 2/2] ✅ Testing features saved: {df_test.shape[0]} samples")
    
    # Memory cleanup
    del features_train, features_test, df_train, df_test
    gc.collect()
    tf.keras.backend.clear_session()
    
    print(f"\n[INFO] ✅ {Batch} completed successfully!\n")

# =========================================
# ⚙️ LOOP THROUGH BATCHES - PROCESS ALL DATA
# =========================================
print("\n" + "="*60)
print("STARTING OPTIMIZED BATCH PROCESSING - FULL DATA (100%)")
print("="*60)

for x in range(1, 16):
    batch_size = 8  # ✅ Optimized batch size
    Batch_Name = f'Batch_{x}'
    
    current_dir = os.path.dirname(os.path.abspath('__file__'))
    parent_dir = os.path.dirname(current_dir)
    root_dir = os.path.dirname(parent_dir)
    
    save_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Results', 'XceptionNet2048+SVM')
    os.makedirs(os.path.join(save_dir, Batch_Name), exist_ok=True)
    
    Training_Path = os.path.join(root_dir, 'pathologyStudentsAug25', 'original', Batch_Name, 'Training')
    Testing_Path = os.path.join(root_dir, 'pathologyStudentsAug25', 'original', Batch_Name, 'Testing')
    
    if not os.path.exists(Training_Path):
        print(f"[WARNING] Skipping {Batch_Name} - path not found: {Training_Path}")
        continue
    
    # Data generators with NO shuffle (ensures proper ordering)
    train_datagen = ImageDataGenerator(rescale=1. / 255)
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    
    train_generator = train_datagen.flow_from_directory(
        Training_Path,
        target_size=(256, 256),
        batch_size=batch_size,
        shuffle=False,
        classes=['Non_Necrosis', 'Necrosis'],
        seed=42
    )
    
    test_generator = test_datagen.flow_from_directory(
        Testing_Path,
        target_size=(256, 256),
        batch_size=batch_size,
        shuffle=False,
        classes=['Non_Necrosis', 'Necrosis'],
        seed=42
    )
    
    # Process batch with optimized function
    XceptionNet2048_Features_Optimized(
        train_generator, test_generator, Batch_Name
    )

print("\n" + "="*60)
print("✅ ALL BATCHES PROCESSED SUCCESSFULLY!")
print("="*60)




STARTING OPTIMIZED BATCH PROCESSING - FULL DATA (100%)
Found 6342 images belonging to 2 classes.
Found 33 images belonging to 2 classes.

[BATCH] Processing Batch_1
[INFO] ✅ Feature extraction model loaded (cached)

[STEP 1/2] Extracting training features...
[INFO] Extracting features from 6342 images in 793 steps...


2025-11-17 19:28:03.261278: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2025-11-17 19:28:03.301321: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300
2025-11-17 19:28:10.984220: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 88.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  [Progress] 10/793 batches processed...
  [Progress] 20/793 batches processed...
  [Progress] 30/793 batches processed...
  [Progress] 40/793 batches processed...
  [Progress] 50/793 batches processed...
  [Progress] 60/793 batches processed...
  [Progress] 70/793 batches processed...
  [Progress] 80/793 batches processed...
  [Progress] 90/793 batches processed...
  [Progress] 100/793 batches processed...
  [Progress] 110/793 batches processed...
  [Progress] 120/793 batches processed...
  [Progress] 130/793 batches processed...
  [Progress] 140/793 batches processed...
  [Progress] 150/793 batches processed...
  [Progress] 160/793 batches processed...
  [Progress] 170/793 batches processed...
  [Progress] 180/793 batches processed...
  [Progress] 190/793 batches processed...
  [Progress] 200/793 batches processed...
  [Progress] 210/793 batches processed...
  [Progress] 220/793 batches processed...
  [Progress] 230/793 batches processed...
  [Progress] 240/793 batches processed...
 

In [10]:
def XceptionNet2048_SVM(train_generator, test_generator, Batch):
  # Get the current notebook's directory and navigate to the correct location
  current_dir = os.path.dirname(os.path.abspath('__file__'))
  parent_dir = os.path.dirname(current_dir)
  root_dir = os.path.dirname(parent_dir)
  
  save_drive_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Results', 'XceptionNet2048+SVM')
  batch_size = 32
  
  model_Xception = Model(base_model_Xception.input, output)
  for layer in base_model_Xception.layers[:-4]:
    layer.trainable = True

  model_Xception_features_2048 = Model(base_model_Xception.input, output_4)

  columns_2048 = []
  for c in range(2048):
    columns_2048.append('feature_'+str(c))

  features_2048_train = model_Xception_features_2048.predict(train_generator)
  df_2048_train = pd.DataFrame(features_2048_train, columns=columns_2048)
  df_2048_train['label'] = train_generator.labels
  df_2048_train['Image_Name'] = train_generator.filenames
  df_2048_train.to_csv(os.path.join(save_drive_dir, Batch, 'XceptionNet2048_Training.csv'))

  features_2048_test = model_Xception_features_2048.predict(test_generator)
  df_2048_test = pd.DataFrame(features_2048_test, columns=columns_2048)
  df_2048_test['label'] = test_generator.labels
  df_2048_test['Image_Name'] = test_generator.filenames
  df_2048_test.to_csv(os.path.join(save_drive_dir, Batch, 'XceptionNet2048_Testing.csv'))

  # Train = pd.read_csv(os.path.join(save_drive_dir, Batch, 'XceptionNet2048_Training.csv'))
  # Test = pd.read_csv(os.path.join(save_drive_dir, Batch, 'XceptionNet2048_Testing.csv'))

  # Train = Train.drop(['Unnamed: 0','Image_Name'], axis=1)
  # Test = Test.drop(['Unnamed: 0','Image_Name'], axis=1)

  # X_train = Train.iloc[:,:-1].values
  # Y_train = Train.iloc[:,-1].values

  # X_test = Test.iloc[:,:-1].values
  # Y_test = Test.iloc[:,-1].values

  # model = SVC(probability=True)
  # model.fit(X_train, Y_train)
  # y_pred = model.predict(X_test)
  # probability = model.predict_proba(X_test)
  # cm = confusion_matrix(Y_test, y_pred)
  # TP = cm[0][0]
  # TN = cm[1][1]
  # FP = cm[0][1]
  # FN = cm[1][0]
  # precision = precision_score(Y_test, y_pred)
  # recall = recall_score(Y_test, y_pred)
  # f1score = f1_score(Y_test, y_pred)
  # accuracy = accuracy_score(Y_test, y_pred)
  # sensitivity = TP/(TP+FN)
  # specificity = TN/(FP+TN)
  # NPV = TN/(TN+FN)
  # PPV = TP/(TP+FP)
  # mcc = matthews_corrcoef(Y_test, y_pred)
  # kappa_score = cohen_kappa_score(Y_test, y_pred)
  # auc_score = roc_auc_score(Y_test, probability[:,1])
  # columns_metrics = ['model_Name','Necrosis','Non_Necrosis','TP','TN','FP','FN','Accuracy','Recall','F1_score','Precision','AUC_ROC','Sensitiviity','Specificity','MCC','Kappa_Score']

  # dff_metrics = pd.DataFrame(columns=columns_metrics)

  # batch_dir = os.path.join(root_dir, 'pathologyStudentsAug25', Batch)
  # Necrosis_samples = len(os.listdir(os.path.join(batch_dir, 'Testing', 'Necrosis')))
  # Non_Necrosis_samples = len(os.listdir(os.path.join(batch_dir, 'Testing', 'Non_Necrosis')))

  # values = ['XceptionNet2048+SVM',Necrosis_samples,Non_Necrosis_samples,TP,TN,FP,FN,accuracy,recall,f1score,precision,auc_score,sensitivity,specificity,mcc,kappa_score]
  # dff_metrics.loc[0] = values
  # dff_metrics.to_csv(os.path.join(save_drive_dir, Batch, 'XceptionNet2048+SVM_results.csv'))

Batch Name Batch_1
Training Path d:\pathologyStudentsAug25\pathologyStudentsAug25\Batch_1\Training
Testing Path d:\pathologyStudentsAug25\pathologyStudentsAug25\Batch_1\Testing
Found 3169 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
  5/100 ━━━━━━━━━━━━━━━━━━━━ 2:16:01 86s/step

In [11]:
for x in range(1,16):
  batch_size = 32
  Batch_Name = f'Batch_{x}'
  print('Batch Name', Batch_Name)
  
  # Get the current notebook's directory and navigate to the correct location
  current_dir = os.path.dirname(os.path.abspath('__file__'))
  parent_dir = os.path.dirname(current_dir)
  root_dir = os.path.dirname(parent_dir)
  
  # Set up paths
  save_dir = os.path.join(root_dir, 'pathologyStudentsAug25', 'Results', 'XceptionNet2048+SVM')
  os.makedirs(os.path.join(save_dir, Batch_Name), exist_ok=True)

  Training_Path = os.path.join(root_dir, 'pathologyStudentsAug25', Batch_Name, 'Training')
  Testing_Path = os.path.join(root_dir, 'pathologyStudentsAug25', Batch_Name, 'Testing')

  print('Training Path', Training_Path)
  print('Testing Path', Testing_Path)
  
  train_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1 / 255.0)

  train_generator = train_datagen.flow_from_directory(
      directory=Training_Path,
      target_size=(256, 256),
      batch_size=batch_size,
      shuffle=False,
      classes=['Non_Necrosis','Necrosis'],
      subset='training',
      seed=42)

  test_generator = test_datagen.flow_from_directory(
      directory=Testing_Path,
      target_size=(256, 256),
      batch_size=batch_size,
      shuffle=False,
      classes=['Non_Necrosis','Necrosis'],
      seed=42
      )
  XceptionNet2048_SVM(train_generator, test_generator, Batch_Name)

Batch Name Batch_1
Training Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_1/Training
Testing Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_1/Testing
Found 3169 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


2025-11-11 11:53:23.299186: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Batch Name Batch_2
Training Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_2/Training
Testing Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_2/Testing
Found 2868 images belonging to 2 classes.
Found 183 images belonging to 2 classes.
90/90 ━━━━━━━━━━━━━━━━━━━━ 19s 205ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step
Batch Name Batch_3
Training Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_3/Training
Testing Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_3/Testing
Found 3204 images belonging to 2 classes.
Found 15 images belonging to 2 classes.
101/101 ━━━━━━━━━━━━━━━━━━━━ 17s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
Batch Name Batch_4
Training Path /home/llmPathoUser/pathologyStudentsAug25/pathologyStudentsAug25/Batch_4/Training
Testing Path /home/llmPathoUser/pathologyStudentsAug25/